In [2]:
%matplotlib inline
from SALib.sample import saltelli
from batchrunner import BatchRunnerMP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from model import BeeEvolutionModel
from agents import *

## OFAT Sensitivity Analysis(Local Analysis)

In [9]:
problem = {
    'num_vars': 3,
    'names': ['alpha', 'forager_royal_ratio', 'growth_factor'],
    'bounds': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
    'nominal_value': [0.5, 0.5, 0.5]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 10 #10
# max_steps = 400 #400
distinct_samples = 20 #40
seed = 20

# Set the outputs
model_reporters = {"Total Workers": lambda m: len([agent for agent in m.schedule_bees_and_flower_patches.agents if isinstance(agent, Worker)]),
                   "Total Queens": lambda m: len([agent for agent in m.schedule_bees_and_flower_patches.agents if isinstance(agent, Queen)]),
                   "Total Drones": lambda m: len([agent for agent in m.schedule_bees_and_flower_patches.agents if isinstance(agent, Drone)]),
                   "Total Fertilized Queens": lambda m: m.get_total_fertilized_queens()}

data = {}

for i, var in enumerate(problem['names']):
    # Get the bounds for this variable and get <distinct_samples> samples within this space (uniform)
    samples = np.linspace(*problem['bounds'][i], num=distinct_samples)
    
    fixed_parameters = {}
    for j, var_ in enumerate(problem['names']):
        if j != i:
            fixed_parameters[var_] = problem['nominal_value'][j]
    fixed_parameters["seed"] = seed
    
    variable_parameters=[{var: sample} for sample in samples]
    
    batch = BatchRunnerMP(BeeEvolutionModel, 
                        fixed_parameters=fixed_parameters,
                        iterations=replicates,
                        variable_parameters=variable_parameters,
                        model_reporters=model_reporters,
                        display_progress=True)
    
    batch.run_all()
    
    data[var] = batch.get_model_vars_dataframe()
#     data[var] = batch.get_collector_model
    

BatchRunner MP will use 8 processors.


200it [28:46,  8.63s/it]

BatchRunner MP will use 8 processors.



200it [28:55,  8.68s/it]

BatchRunner MP will use 8 processors.



200it [25:21,  7.61s/it]


In [4]:
data

{'alpha':       alpha  Run  Total Drones  Total Fertilized Queens  Total Queens  \
 0  0.222222  0.5             1                        0             0   
 1  0.666667  0.5             1                        0             0   
 2  0.111111  0.5             1                        0             0   
 3  0.555556  0.5             1                        0             0   
 4  0.444444  0.5             1                        0             0   
 5  0.000000  0.5             1                        0             0   
 6  0.333333  0.5             1                        0             0   
 7  0.777778  0.5             1                        0             0   
 8  1.000000  0.5             1                        0             0   
 9  0.888889  0.5             1                        0             0   
 
    Total Workers  forager_royal_ratio  growth_factor  seed  daily_steps  
 0              0                  0.5            0.5    20            4  
 1              0        